---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

text  target
0  Go until jurong point, crazy.. Available only ...       0
1                      Ok lar... Joking wif u oni...       0
2  Free entry in 2 a wkly comp to win FA Cup fina...       1
3  U dun say so early hor... U c already then say...       0
4  Nah I don't think he goes to usf, he lives aro...       0
5  FreeMsg Hey there darling it's been 3 week's n...       1
6  Even my brother is not like to speak with me. ...       0
7  As per your request 'Melle Melle (Oru Minnamin...       0
8  WINNER!! As a valued network customer you have...       1
9  Had your mobile 11 months or more? U R entitle...       1

In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def answer_one():
    ratio = len(spam_data[spam_data['target']==1]) / len(spam_data)
    
    return ratio*100

In [4]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    vocab = CountVectorizer().fit(X_train).vocabulary_
    vocab = [x for x in vocab.keys()]
    len_vocab = [len(x) for x in vocab]
    
    return vocab[np.argmax(len_vocab)]

In [6]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    vector = CountVectorizer().fit(X_train)
    
    X_train_new = vector.transform(X_train)
    X_test_new = vector.transform(X_test)
    
    clf = MultinomialNB(alpha=0.1)
    clf.fit(X_train_new, y_train)
    
    y_predicted = clf.predict(X_test_new)
    
    return roc_auc_score(y_test, y_predicted)

In [9]:
answer_three()

0.97208121827411165

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    import operator
    vect = TfidfVectorizer()
    X_train_new = vect.fit_transform(X_train)
    
    vect_features = vect.get_feature_names()
    
    idfs = vect.idf_
    names_idfs = list(zip(vect_features, idfs))

    smallest_tf_idfs_series = sorted(names_idfs, key=operator.itemgetter(1))[:20]
    smallest_tf_idfs_series = pd.Series([features[1] for features in smallest_tf_idfs_series], index=[features[0] for features in smallest_tf_idfs_series])

    largest_tf_idfs_series = sorted(names_idfs, key=operator.itemgetter(1), reverse=True)[:20]
    
    largest_tf_idfs_series = sorted(largest_tf_idfs_series, key=operator.itemgetter(0))
    largest_tf_idfs_series = pd.Series([features[1] for features in largest_tf_idfs_series], index=[features[0] for features in largest_tf_idfs_series])
    return (smallest_tf_idfs_series, largest_tf_idfs_series) 

In [11]:
answer_four()

(to      2.198406
 you     2.265645
 the     2.707383
 in      2.890761
 and     2.976764
 is      3.003012
 me      3.111530
 for     3.206840
 it      3.222174
 my      3.231044
 call    3.297812
 your    3.300196
 of      3.319473
 have    3.354130
 that    3.408477
 on      3.463136
 now     3.465949
 can     3.545053
 are     3.560414
 so      3.566625
 dtype: float64, 000pes         8.644919
 0089           8.644919
 0121           8.644919
 01223585236    8.644919
 0125698789     8.644919
 02072069400    8.644919
 02073162414    8.644919
 02085076972    8.644919
 021            8.644919
 0430           8.644919
 07008009200    8.644919
 07099833605    8.644919
 07123456789    8.644919
 0721072        8.644919
 07753741225    8.644919
 077xxx         8.644919
 078            8.644919
 07808247860    8.644919
 07808726822    8.644919
 078498         8.644919
 dtype: float64)

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [12]:
def answer_five():
    vect = TfidfVectorizer(min_df=3)
    
    X_train_new = vect.fit_transform(X_train)
    X_test_new = vect.transform(X_test)
    
    clf = MultinomialNB(alpha = 0.1)
    clf.fit(X_train_new, y_train)
    
    y_predicted = clf.predict(X_test_new)
    
    return roc_auc_score(y_test, y_predicted)

In [13]:
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [14]:
def answer_six():
    avg_len_spam = [len(x) for x in spam_data.loc[spam_data['target']==1, 'text']]
    avg_len_not_spam = [len(x) for x in spam_data.loc[spam_data['target']==0, 'text']]
    return (np.mean(avg_len_not_spam), np.mean(avg_len_spam))

In [15]:
answer_six()

(71.023626943005183, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [16]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [17]:
from sklearn.svm import SVC

def answer_seven():
    vect = TfidfVectorizer(min_df=5)
    X_train_new = vect.fit_transform(X_train)
    X_train_new_len = add_feature(X_train_new, X_train.str.len())
    
    X_test_new = vect.transform(X_test)
    X_test_new_len = add_feature(X_test_new, X_test.str.len())
    
    clf = SVC(C=10000)
    
    clf.fit(X_train_new_len, y_train)
    y_predicted = clf.predict(X_test_new_len)
    
    return roc_auc_score(y_test, y_predicted)

In [18]:
answer_seven()

0.95813668234215565

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [19]:
def answer_eight():
    spam = [sum(char.isnumeric() for char in x) for x in spam_data.loc[spam_data['target']==1,'text']]
    not_spam = [sum(char.isnumeric() for char in x) for x in spam_data.loc[spam_data['target']==0,'text']]
    return (np.mean(spam),np.mean(not_spam))

In [20]:
answer_eight()

(15.76037483266399, 0.29927461139896372)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [21]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    train_new = [sum(char.isnumeric() for char in x) for x in X_train]
    test_new = [sum(char.isnumeric() for char in x) for x in X_test]
    
    tf = TfidfVectorizer(min_df = 5, ngram_range = (1,3)).fit(X_train)
    X_train_new = tf.transform(X_train)
    X_test_new = tf.transform(X_test)
    
    X_train_new = add_feature(X_train_new, train_new)
    X_test_new = add_feature(X_test_new, test_new)
    
    clf = LogisticRegression(C=100).fit(X_train_new, y_train)
    pred = clf.predict(X_test_new)
    
    return roc_auc_score(y_test, pred)

In [22]:
answer_nine()

0.96787090640544626

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [23]:
def answer_ten():
    
    return (np.mean(spam_data.loc[spam_data['target']==0,'text'].str.count('\W')), 
            np.mean(spam_data.loc[spam_data['target']==1,'text'].str.count('\W')))

In [24]:
answer_ten()

(17.291813471502589, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [25]:
def answer_eleven():
    
    len_train = [len(x) for x in X_train]
    len_test = [len(x) for x in X_test]
    train_new = [sum(char.isnumeric() for char in x) for x in X_train]
    test_new = [sum(char.isnumeric() for char in x) for x in X_test]
    
    nan_train = X_train.str.count('\W')
    nan_test = X_test.str.count('\W')
    
    cv = CountVectorizer(min_df = 5, ngram_range=(2,5), analyzer='char_wb').fit(X_train)
    X_train_new = cv.transform(X_train)
    X_test_new = cv.transform(X_test)
    
    X_train_new = add_feature(X_train_new, [len_train, train_new, nan_train])
    X_test_new = add_feature(X_test_new, [len_test, test_new, nan_test])
    
    clf = LogisticRegression(C=100).fit(X_train_new, y_train)
    pred = clf.predict(X_test_new)
    
    auc_score = roc_auc_score(y_test, pred)
    
    feature_names = np.array(cv.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count'])
    sorted_coef_index = clf.coef_[0].argsort()
    smallest_coefs = feature_names[sorted_coef_index[:10]]
    largest_coefs = feature_names[sorted_coef_index[:-11:-1]]
    
    return (auc_score, list(smallest_coefs), list(largest_coefs))

In [26]:
answer_eleven()

(0.97885931107074342,
 ['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'],
 ['digit_count', 'ne', 'ia', 'co', 'xt', ' ch', 'mob', ' x', 'ww', 'ar'])